# Generate the size value

In [ ]:
# Load images
for filename in os.listdir(dataset_path):
    if filename.endswith("_piece_virtual_mask.png"):
        meta_filename = filename[:-23]
        print(meta_filename)

        ar_region_mask = cv2.imread(dataset_path+"/"+meta_filename+"_piece_virtual_mask.png", cv2.IMREAD_GRAYSCALE) 
        ar_region_mask = (ar_region_mask > 128).astype(np.uint8)

        ar_depth_raw = np.load(dataset_path+"/"+meta_filename+"_ar_depth.npy")
        #ar_depth_raw = np.load(dataset_path+"/size_depth_array_"+meta_filename+"_ar.npy")
        mask_virtual_depth = ar_depth_raw[ar_region_mask > 0]
        size = np.max(mask_virtual_depth)-np.min(mask_virtual_depth)
        print(meta_filename, "size: ", str(size))
        print(type(df["filename"].tolist()[0]))

        # Update the size column where the filename matches
        csv_mask = df["filename"] == meta_filename
        if csv_mask.any():
            df.loc[csv_mask, "size"] = size  # Update the 'size' column
            df.to_csv(csv_file, index=False)  # Save back to CSV
            print(f"Updated size for {meta_filename} -> {size}")
        else:
            print(f"Filename {meta_filename} not found in CSV.")

# Generate the clipping value

In [ ]:
def count_white_areas(binary_image):
    # Compute connected components. 
    # Note: The function returns the total number of labels, where label 0 is the background.
    num_labels, labels = cv2.connectedComponents(binary_image)
    # Number of closed white regions is total labels minus one (background)
    return num_labels - 1

def get_mean_depth_per_region(binary_mask, depth_array, t=0.01):
    """
    Computes the mean depth value of each white area in the binary mask.

    Parameters:
        binary_mask (numpy.ndarray): A binary mask where white areas (255) represent regions of interest.
        depth_array (numpy.ndarray): A depth map of the same shape as the binary mask.

    Returns:
        list of tuples: Each tuple contains (label, mean_depth_value).
    """
    # Label connected components
    num_labels, labels = cv2.connectedComponents(binary_mask)

    mean_depth_values = []
    new_mask = np.zeros_like(binary_mask)

    for label in range(1, num_labels):  # Skip background (label 0)
        region_mask = labels == label  # Boolean mask for this region
        mean_depth = depth_array[region_mask].mean()  # Compute mean depth
        #print(mean_depth)
        mean_depth_values.append((label, mean_depth))
        if mean_depth > t:
            new_mask[region_mask] = 1  # Keep this region

    return mean_depth_values, new_mask

for filename in os.listdir(dataset_path):
    if filename.endswith("_full_virtual_mask.png"):
        meta_filename = filename[:-22]
        print(meta_filename)

        if "hl_" in meta_filename:
            ar_img = cv2.imread(dataset_path+"/"+meta_filename+"_ar.jpg")
        else:
            ar_img = cv2.imread(dataset_path+"/"+meta_filename+"_ar.png")
        ar_img = cv2.cvtColor(ar_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        ar_region_mask = cv2.imread(dataset_path+"/"+meta_filename+"_piece_virtual_mask.png", cv2.IMREAD_GRAYSCALE) 
        ar_region_mask = (ar_region_mask > 128).astype(np.uint8)


        raw_depth = np.load(dataset_path+"/"+meta_filename+"_raw_depth.npy")
        #raw_depth = np.load(dataset_path+"/size_depth_array_"+meta_filename+"_raw.npy")
        raw_depth = (raw_depth - raw_depth.min()) / (raw_depth.max() - raw_depth.min())
        ar_depth = np.load(dataset_path+"/"+meta_filename+"_ar_depth.npy")
        #ar_depth = np.load(dataset_path+"/size_depth_array_"+meta_filename+"_ar.npy")
        ar_depth = (ar_depth - ar_depth.min()) / (ar_depth.max() - ar_depth.min())

        diff_depth = abs(raw_depth - ar_depth)
        mean_depth_values, new_mask = get_mean_depth_per_region(ar_region_mask, diff_depth, t=0.08)
        
        new_count = count_white_areas(new_mask)
        if new_count==0:
            new_count = count_white_areas(ar_region_mask)

        csv_mask = df["filename"] == meta_filename
        if csv_mask.any():
            df.loc[csv_mask, "clipping"] = new_count  # Update the 'size' column
            df.to_csv(csv_file, index=False)  # Save back to CSV
            print(f"Updated new_count for {meta_filename} -> {new_count}")
        else:
            print(f"Filename {meta_filename} not found in CSV.")
        '''
        plt.figure(figsize=(10, 7))
        plt.subplot(2, 2, 1)
        plt.imshow(ar_img)
        print(mean_depth_values)
        plt.title('Original ar image')
        plt.subplot(2, 2, 2)
        plt.imshow(ar_region_mask)
        plt.title('mask image have '+str(count_white_areas(ar_region_mask)) + " masks")
        plt.subplot(2, 2, 3)
        plt.imshow(ar_depth)
        plt.title('AR Depth Map')
        plt.colorbar()
        plt.subplot(2, 2, 4)
        plt.imshow(new_mask)
        plt.title('new mask image have '+str(count_white_areas(new_mask)) + " masks")
        plt.colorbar()
        '''

# Generate the floating value

In [ ]:
def get_obvious_region(binary_mask, depth_array, t=0.01):
    """
    Computes the mean depth value of each white area in the binary mask.

    Parameters:
        binary_mask (numpy.ndarray): A binary mask where white areas (255) represent regions of interest.
        depth_array (numpy.ndarray): A depth map of the same shape as the binary mask.

    Returns:
        list of tuples: Each tuple contains (label, mean_depth_value).
    """
    # Label connected components
    num_labels, labels = cv2.connectedComponents(binary_mask)

    new_mask = np.zeros_like(binary_mask)

    for label in range(1, num_labels):  # Skip background (label 0)
        region_mask = labels == label  # Boolean mask for this region
        mean_depth = depth_array[region_mask].mean()  # Compute mean depth
        if mean_depth > t:
            new_mask[region_mask] = 1  # Keep this region

    return new_mask

def is_object_floating(ar_img, depth_array, binary_mask, bottom_margin=10, below_margin=10, threshold=10, min_contact_ratio=0.1):
    """
    Determines whether an object is floating, accounting for partial support.

    Parameters:
        depth_array (numpy array): The depth data in (w, h) format.
        binary_mask (numpy array): The binary mask of the object.
        bottom_margin (int): Pixels from the bottom to consider for contact detection.
        threshold (float): Depth difference threshold to classify as floating.
        min_contact_ratio (float): Minimum fraction of the bottom in contact to consider the object as "resting".

    Returns:
        bool: True if the object is floating, False otherwise.
    """
    object_indices = np.argwhere(binary_mask > 0)
    if len(object_indices) == 0:
        return -1, -1, -1  # No object found

    # Find bottom-most row of the object
    max_y = np.max(object_indices[:, 0])

    # Select the bottom part of the object (a few pixels above max_y)
    bottom_mask = np.zeros_like(binary_mask)
    bottom_region = (object_indices[:, 0] >= max_y - bottom_margin)
    bottom_indices = object_indices[bottom_region]
    bottom_mask[bottom_indices[:, 0], bottom_indices[:, 1]] = 1
    # Extract depth of the bottom region of the object
    bottom_depths = depth_array[bottom_mask > 0]

    # Get background depth just below the object
    background_mask = np.zeros_like(binary_mask)
    # Check pixels from 1 to `below_margin` pixels below the object
    for shift in range(1, below_margin + 1):
        below_indices = bottom_indices + np.array([shift, 0])  # Shift downward
        valid_below_indices = below_indices[:, 0] < depth_array.shape[0]  # Ensure within bounds
        below_indices = below_indices[valid_below_indices]
        background_mask[below_indices[:, 0], below_indices[:, 1]] = 1
    background_mask[object_indices[:, 0], object_indices[:, 1]] = 0
    # Extract depth of the bottom background region of the object
    background_depths = depth_array[background_mask > 0]

    # Compute median depth values
    if len(bottom_depths) == 0 or len(background_depths) == 0:
        print("Not enough data", len(bottom_depths), len(background_depths))
        return -1, -1, -1  # Not enough data

    object_bottom_median = np.mean(bottom_depths)
    background_median = np.mean(background_depths)

    # Detect floating based on depth difference
    floating_condition = object_bottom_median - background_median > threshold

    contact_mask = (bottom_mask > 0) & (abs(depth_array-background_median) <= threshold)  # Small depth difference = contact

    # Label connected contact regions
    labeled_contact, num_labels = label(contact_mask)

    # Find the largest contact region
    largest_contact_area = max(np.bincount(labeled_contact.flat)[1:], default=0)  # Ignore background label (0)

    # Compute ratio of contact points to total bottom pixels
    total_bottom_pixels = np.count_nonzero(bottom_mask)
    contact_ratio = largest_contact_area / total_bottom_pixels if total_bottom_pixels > 0 else 0

    # Final decision: if most of the bottom is unsupported, it's floating
    is_floating = floating_condition or (contact_ratio < min_contact_ratio)

    original_img = ar_img
    original_img[:,:,0] = depth_array * 255.0
    original_img[:,:,1] = depth_array * 255.0
    original_img[:,:,2] = depth_array * 255.0
    original_img = original_img.astype(float) / 255.0  # Normalize to 0-1 range

    # Create colored overlay masks
    overlay = np.zeros_like(original_img)

    # Define colors (in RGB)
    bottom_color = np.array([1, 0, 0])  # Red for bottom region
    background_color = np.array([0, 1, 0])  # Green for background region
    contact_color = np.array([0, 0, 1])  # Blue for contact region

    # Apply color to mask regions
    for i in range(3):  # Apply to each RGB channel
        overlay[:, :, i] += bottom_mask * bottom_color[i]  # Red overlay
        overlay[:, :, i] += background_mask * background_color[i]  # Green overlay
        overlay[:, :, i] += contact_mask * contact_color[i]  # Blue overlay

    # Blend overlay with original image
    alpha = 0.5
    blended_img = (1 - alpha) * original_img + alpha * overlay

    # Ensure values are in valid range
    blended_img = np.clip(blended_img, 0, 1)  # Keep values between 0 and 1
    blended_img = (blended_img * 255).astype(np.uint8)  # Convert back to 0-255 range
    # Show the result
    '''
    plt.figure(figsize=(16, 12))
    plt.imshow(blended_img)
    plt.title("Bottom Mask (Red) & Background Mask (Green) Over Original Image")
    plt.axis("off")
    plt.show()
    '''
    return object_bottom_median - background_median, contact_ratio, is_floating

for filename in os.listdir(dataset_path):
    if filename.endswith("_full_virtual_mask.png"):
        meta_filename = filename[:-22]
        print(meta_filename)

        if "hl_" in meta_filename:
            ar_img = cv2.imread(dataset_path+"/"+meta_filename+"_ar.jpg")
        else:
            ar_img = cv2.imread(dataset_path+"/"+meta_filename+"_ar.png")
        
        ar_img = cv2.cvtColor(ar_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        ar_region_mask = cv2.imread(dataset_path+"/"+meta_filename+"_full_virtual_mask.png", cv2.IMREAD_GRAYSCALE) 
        ar_region_mask = (ar_region_mask > 128).astype(np.uint8)

        raw_depth = np.load(dataset_path+"/"+meta_filename+"_raw_depth.npy")
        #raw_depth = np.load(dataset_path+"/depth_array_"+meta_filename+"_raw.npy")
        raw_depth = (raw_depth - raw_depth.min()) / (raw_depth.max() - raw_depth.min())
        ar_depth_origin = np.load(dataset_path+"/"+meta_filename+"_ar_depth.npy")
        #ar_depth_origin = np.load(dataset_path+"/depth_array_"+meta_filename+"_ar.npy")
        ar_depth = (ar_depth_origin - ar_depth_origin.min()) / (ar_depth_origin.max() - ar_depth_origin.min())

        diff_depth = abs(raw_depth - ar_depth)
        new_mask = get_obvious_region(ar_region_mask, diff_depth, t=0.08)
        
        diff, contact_ratio, floating = is_object_floating(ar_img, ar_depth_origin, new_mask, bottom_margin=40, below_margin=30, threshold=0.1, min_contact_ratio=0.2)
        print("Is the object floating?", diff, contact_ratio, floating) 

        csv_mask = df["filename"] == meta_filename
        if csv_mask.any():
            df.loc[csv_mask, "floating_depth_diff"] = diff  
            df.loc[csv_mask, "floating_contact_ratio"] = contact_ratio
            df.to_csv(csv_file, index=False)  # Save back to CSV
            print(f"Updated floating for {meta_filename} -> {diff}, {contact_ratio}")
        else:
            print(f"Filename {meta_filename} not found in CSV.")

# Generate the brightness value

In [ ]:
def compute_average_luminosity_numpy(image_path):
    """
    Computes the average luminosity (brightness) of an image using the weighted method:
       Luminosity = 0.299*R + 0.587*G + 0.114*B
    Returns a float value representing the average luminosity (0-255 for 8-bit).
    """
    # Read image into NumPy array (H x W x 3)
    img = np.array(Image.open(image_path).convert("RGB"), dtype=float)
    
    # Separate channels
    R = img[..., 0]
    G = img[..., 1]
    B = img[..., 2]
    
    # Compute luminosity per pixel
    luminosity = 0.299 * R + 0.587 * G + 0.114 * B
    
    # Return the average
    return np.mean(luminosity)

def order_images_by_luminosity(directory_path):
    """
    1. Find files in `directory_path`
    2. Compute their average luminosity
    """

    meta_filenames_dic = {}
    for filename in os.listdir(directory_path):
        if "cropped" in filename and "background" not in filename:
        #if ".png" in filename and "_" in filename:
            meta_filename = filename.split("_cropped_")[0]  # Extract only the first part
            #meta_filename = filename.split("_")[0]
            if meta_filename not in meta_filenames_dic.keys():
                meta_filenames_dic[meta_filename] = {"real_luminosities":[], "virtual_luminosity": -1}
            file_path = os.path.join(directory_path, filename)
            # 2. Compute luminosity
            try:
                lum = compute_average_luminosity_numpy(file_path)
                if "virtual" in filename:
                    meta_filenames_dic[meta_filename]["virtual_luminosity"] = lum
                else:
                    meta_filenames_dic[meta_filename]["real_luminosities"].append(lum)
            except Exception as e:
                print(f"Could not process '{filename}': {e}")
    
    for meta_filename in meta_filenames_dic.keys():
        csv_mask = df["filename"] == meta_filename
        print(meta_filename)
        if csv_mask.any():
            if len(meta_filenames_dic[meta_filename]["real_luminosities"]) == 0:
                brightness_min = meta_filenames_dic[meta_filename]["virtual_luminosity"]
                brightness_max = meta_filenames_dic[meta_filename]["virtual_luminosity"]
            else:
                brightness_min = np.min(meta_filenames_dic[meta_filename]["real_luminosities"])
                brightness_max = np.max(meta_filenames_dic[meta_filename]["real_luminosities"])
            brightness_virtual = meta_filenames_dic[meta_filename]["virtual_luminosity"]
            df.loc[csv_mask, "brightness_min"] = brightness_min
            df.loc[csv_mask, "brightness_max"] = brightness_max
            df.loc[csv_mask, "brightness_virtual"] = brightness_virtual
            df.to_csv(csv_file, index=False)  # Save back to CSV
            print(f"Updated brightness for {meta_filename} -> {brightness_min}, {brightness_max}, {brightness_virtual}")
        else:
            print(f"Filename {meta_filename} not found in CSV.")

order_images_by_luminosity(dataset_path)

# Generate the light direction value

In [ ]:
import pandas as pd

# List of Excel files to combine
xlsx_files = ['ARQA_lightdir.xlsx', 'ARpatches_lightdir.xlsx', 'cropped_real_lightdir.xlsx']

# Read and combine all Excel files
combined_df = pd.concat([pd.read_excel(file) for file in xlsx_files], ignore_index=True)

# Save to CSV
combined_df.to_csv('light_direction_results.csv', index=False)

In [ ]:
light_csv_file = "light_direction_results.csv"
light_df = pd.read_csv(light_csv_file, dtype={"filename": str})

for meta_filename in meta_filenames:
    light_direction_real_list = light_df.loc[light_df["filename"].str.contains(meta_filename+"_cropped", na=False) 
                                             & ~light_df["filename"].str.contains("virtual", na=False) 
                                             & ~light_df["filename"].str.contains("background", na=False), "results"].tolist()
    light_direction_virtual = light_df.loc[light_df["filename"].str.contains(meta_filename+"_cropped_virtual", na=False), "results"].values[0]
    light_direction_whole = light_df.loc[light_df["filename"] == meta_filename+"_raw", "results"].values[0]

    csv_mask = df["filename"] == meta_filename
    if csv_mask.any():
        df.loc[csv_mask, "light_direction_real_list"] = light_direction_real_list
        df.loc[csv_mask, "light_direction_virtual"] = light_direction_virtual
        df.loc[csv_mask, "light_direction_whole"] = light_direction_whole
        df.to_csv(csv_file, index=False)  # Save back to CSV
        print(f"Updated light direction for {meta_filename} -> {light_direction_real_list}, {light_direction_whole}, {light_direction_virtual}")
    else:
        print(f"Filename {meta_filename} not found in CSV.")

# Generate the shadow value

In [ ]:
import pandas as pd

# List of Excel files to combine
xlsx_files = ['ARQA_confidence_score.xlsx', 'ARpatches_confidence_score.xlsx', 'cropped_real_confidence_score.xlsx']

# Read and combine all Excel files
combined_df = pd.concat([pd.read_excel(file) for file in xlsx_files], ignore_index=True)

# Save to CSV
combined_df.to_csv('shadow_results.csv', index=False)

In [ ]:
shadow_csv_file = "shadow_results.csv"
shadow_df = pd.read_csv(shadow_csv_file, dtype={"filename": str})

for meta_filename in meta_filenames:
    shadow_real_list = shadow_df.loc[shadow_df["filename"].str.contains(meta_filename+"_cropped", na=False) 
                                             & ~shadow_df["filename"].str.contains("virtual", na=False) 
                                             & ~shadow_df["filename"].str.contains("background", na=False), "results"].tolist()
    shadow_virtual = shadow_df.loc[shadow_df["filename"].str.contains(meta_filename+"_cropped_virtual", na=False), "results"].values[0]
    shadow_whole = shadow_df.loc[shadow_df["filename"] == meta_filename+"_raw", "results"].values[0]

    csv_mask = df["filename"] == meta_filename
    if csv_mask.any():
        df.loc[csv_mask, "shadow_real_list"] = light_direction_real_list
        df.loc[csv_mask, "shadow_virtual"] = shadow_virtual
        df.loc[csv_mask, "shadow_whole"] = shadow_whole
        df.to_csv(csv_file, index=False)  # Save back to CSV
        print(f"Updated shadow for {meta_filename} -> {shadow_real_list}, {shadow_whole}, {shadow_virtual}")
    else:
        print(f"Filename {meta_filename} not found in CSV.")

# Get the rule-based score

In [ ]:
# Function to initialize the CSV file with 10 filenames
def initialize_rule_csv(csv_file, filenames):
    csv_columns = [
        "filename", 
        "visual_coherence", "size", "clipping", "floating", "obstruction", 
        "brightness", "light_direction", "shadow"
    ]
    # If the CSV doesn't exist or is empty, create it with default values
    if not os.path.exists(csv_file) or os.stat(csv_file).st_size == 0:
        df = pd.DataFrame(columns=csv_columns)  # Create empty DataFrame with correct columns
        
        # Add 10 filenames with "N/A" for all other columns
        df["filename"] = filenames
        for col in csv_columns[1:]:  # Set all other columns to "N/A"
            df[col] = "N/A"

        df.to_csv(csv_file, index=False)
        print(f"Initialized CSV with 10 filenames: {csv_file}")
    else:
        print(f"CSV already exists and is not empty: {csv_file}")

rule_csv_file = "Testing_rule_based_scores.csv"
print(meta_filenames)
initialize_rule_csv(rule_csv_file, list(meta_filenames))

rule_df = pd.read_csv(rule_csv_file, dtype={"filename": str})

size_typical_range_dic = {
        "scissors": [15, 25],
        "knife": [15, 30],
        "stop_sign": [76, 90],
        "candle": [8, 30],
        "baby": [45, 60],
        "person": [150, 180],
        "shoes": [23, 30],
        "basketball": [24, 25],
        "football": [28, 30],
        "chair": [40, 100],  # Seat height ~40 cm, full height ~100 cm
        "table": [70, 150],  # Small ~70 cm, large dining ~150 cm
        "apple": [8, 10],
        "toy": [5, 100],  # Varies widely
        "box": [10, 100],  # Small ~10 cm, large ~100 cm
        "cupnoodle": [10, 12],
        "pillow": [30, 80],  # Small ~30 cm, large body pillow ~80 cm
        "handglove": [20, 25],
        "balloon": [20, 50],  # Deflated ~20 cm, large inflated ~50 cm
        "basket": [20, 60],  # Small ~20 cm, large ~60 cm
        "can": [12, 15],
        "car": [350, 500],  # Small car ~350 cm, large car ~500 cm
        "painting": [30, 150],  # Small ~30 cm, large wall painting ~150 cm
        "skull": [20, 25],
        "brain": [14, 17],
        "eye": [2, 2.5],
        "heart": [10, 15],
        "leg": [2.5, 40],  # Small ~10 cm, femur ~50 cm
        "liver": [10, 23],  # Varies by type
        "remote_controller": [15, 25],
        "laptop": [30, 45],  # Thin laptops ~30 cm, large ~45 cm
        "keyboard": [43, 50],  # Standard ~43 cm, extended ~50 cm
        "desktop": [38, 60],  # Varies by case size
        "bottle": [20, 35],  # Small ~20 cm, large ~35 cm
        "pot": [15, 40],  # Small saucepan ~15 cm, large stockpot ~40 cm
        "bowl": [10, 30],  # Small ~10 cm, large ~30 cm
        "tissue_box": [20, 30],
        "cat": [30, 90],  # Small kittens ~30 cm, large breeds ~90 cm (nose to tail)
        "door": [200, 210],
        "desk": [70, 180],  # Small ~70 cm, large work desk ~180 cm
        "cup": [8, 15],
        "book": [20, 40],  # Small ~20 cm, large coffee table book ~40 cm
        "lamp": [38, 180],  # Small table lamps ~38 cm, large floor lamps ~180 cm
        "backpack": [10, 70]
    }

In [ ]:
import re
import ast
def get_size_score(meta_filename, size):
    size_cm = size * 100.0
    print(size_cm)
    object_class = meta_filename.split("_")[1].lower()
    object_class = re.sub(r'\d+', '', object_class)
    object_class = object_class.replace("wired", "")
    object_class = object_class.replace("lobester", "")
    object_class = object_class.replace("pepsi", "can")
    typical_min = size_typical_range_dic[object_class][0]
    typical_max = size_typical_range_dic[object_class][1]

    score = -1
    tmp = -1
    if size_cm > typical_max:
        tmp = (size_cm - typical_max)/typical_max
    elif size_cm < typical_min:
        tmp = (typical_min - size_cm)/typical_min
    else:
        score = 5

    if score == -1:
        if tmp <= 0.2:
            score = 5
        elif tmp > 0.2 and tmp <= 0.4:
            score = 4
        elif tmp > 0.4 and tmp <= 1.5:
            score = 3
        elif tmp > 1.5 and tmp <= 2.0:
            score = 2
        else:
            score = 1
    

    return score

def get_clipping_score(clipping_area_cnt):
    score = -1
    if clipping_area_cnt == 1:
        score = 5
    elif clipping_area_cnt > 2:
        score = 1
    elif clipping_area_cnt == 2:
        score = 2  
    else:
        score = random.randint(3, 4)
    return score

def get_floating_score(depth_diff, contact_ratio):
    score = -1
    if depth_diff <= 0.01:
        score = 5
    elif depth_diff > 0.01 and depth_diff <= 0.05:
        score = 4
    elif depth_diff > 0.05 and depth_diff <= 0.2:
        score = 3
    elif depth_diff > 0.2 and depth_diff <= 0.3:
        score = 2
    else:
        score = 1
    
    
    if contact_ratio >= 0.8:
        score = 5
    elif contact_ratio >= 0.4 and contact_ratio < 0.6:
        score = min(score+1, 5)
    elif contact_ratio >= 0.2 and contact_ratio < 0.4:
        score = score
    else:
        score = 1
    
    if depth_diff==-1 and contact_ratio==-1:
        score = 3
    return score

def get_obstruction_score(meta_filename, percetage, strength, coverage):
    obstruction_csv_file = "shadow_light_obstruct_estimated_values/obstruction_estimated_values.csv"
    obstruction_df = pd.read_csv(obstruction_csv_file, dtype={"filename": str})

    score = obstruction_df.loc[obstruction_df["filename"] == meta_filename, "pred_score"].values[0]

    return score

def get_brightness_score(meta_filename, min, max, virtual):
    score = -1
    tmp = -1
    if max == min:
        max = np.max([max, 100])
        min = np.min([min, 100])
    if virtual > max:
        tmp = (virtual - max)/(max-min)
    elif virtual < min:
        tmp = (min - virtual)/(max-min)
    else:
        score = 5

    if score == -1:
        if tmp <= 0.5:
            score = 4
        elif tmp > 0.5 and tmp <= 0.7:
            score = 3
        elif tmp > 0.7 and tmp <= 1.0:
            score = 2
        else:
            score = 1
    return score

def get_light_direction_score(real_list, whole, virtual):
    score = -1
    real_list = ast.literal_eval(real_list)
    
    value_whole = whole == virtual
    if len(real_list) == 0:
        value_single = 1
    else:
        value_single = real_list.count(virtual)/len(real_list)
    ratio = 0.5*value_single + 0.5*value_whole

    if ratio<0.2:
        score = 1
    elif ratio>=0.2 and ratio<0.25:
        score = 2
    elif ratio>=0.25 and ratio<0.7:
        score = 3
    elif ratio>=0.7 and ratio<0.75:
        score = 4
    else:
        score = 5
    return score

def get_shadow_score(real_list, whole, virtual):
    score = -1
    real_list = ast.literal_eval(real_list)
    print("shadow real list",real_list)
    if len(real_list)==0:
        return 4
    
    real_max = np.max(real_list)
    real_min = np.min(real_list)
    if virtual >= real_max:
        score = 5
    elif virtual < real_min:
        score = 1
    else:
        virtual_norm = virtual
        if real_min != real_max:
            virtual_norm = (virtual-real_min)/(real_max - real_min)
        if virtual_norm < 0.1:
            score = 1
        elif virtual_norm >= 0.1 and virtual_norm < 0.6: 
            score = 2
        elif virtual_norm >= 0.6 and virtual_norm < 0.7: 
            score = 3
        elif virtual_norm >= 0.7 and virtual_norm < 1.0: 
            score = 4
        else: 
            score = 5
    return score